In [ ]:
from ipywidgets import interact
import numpy as np
from bokeh.layouts import row, column
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d
import time

from bokeh.palettes import Dark2_5 as palette
# itertools handles the cycling
import itertools  
output_notebook()

In [ ]:
import silayer

In [ ]:
hist_fig = figure(title='ASIC_<x>', tools='', 
                  plot_height=350, plot_width=800,
                  background_fill_color="#fafafa", )

hist_fig.xaxis.axis_label = "adc counts (LSB)"

rate_fig = figure(title='event rate', tools='',
                  plot_height=350, plot_width=800,
                  background_fill_color="#fafafa", )                  

bin_edges = np.array(range(1025)) - 0.5
#print(bin_edges)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
#print(bin_centers)
#print(len(bin_edges))
#print(len(bin_centers))

binned_data = np.zeros(len(bin_centers))

hdict = {}
for ch in range(3):
    hdict[f'binned_ch{ch:02d}'] = np.zeros(len(bin_centers))

hdict['bin_centers'] = bin_centers
hdict['left'] = bin_edges[0:-1]
hdict['right'] = bin_edges[1::]
hdict['bottom'] = np.zeros(len(bin_centers))

h_ds = ColumnDataSource(hdict)
#print(h_ds.data)
# create a color iterator
colors = itertools.cycle(palette) 

for ch in range(3):
    hist_fig.quad(top=f'binned_ch{ch:02d}', left='left', right='right', bottom='bottom', 
                  source=h_ds, color=next(colors), legend_label=f'Ch{ch:02d}')
    

rate_ds = ColumnDataSource({'t':[], 'rate':[]})

rate_fig.line('t', 'rate', source=rate_ds)
rate_fig.xaxis.axis_label = "t (s)"
rate_fig.yaxis.axis_label = "event rate (Hz)"

h = show(column(hist_fig, rate_fig), notebook_handle=True)

In [ ]:
#        datastream.stream({'time': [dp.event_counter], 
#                           'adc0':[ap.data[0]], 
#                           'adc1':[ap.data[1]]}, 
#                           rollover=1000)    

def bin_data(bin_edges, datum):
    for i in range(len(bin_edges) - 1):
        if datum >= bin_edges[i] and datum < bin_edges[i+1]: 
            binned_data[i] += 1
            #print(datum, i, binned_data[i])
            return datum, i, binned_data[i]
        else:
            continue

    #should never get here
    print(f"dataum {datum} not in bin limits.")
t0 = time.time()
tlast = time.time()    
ch = 0
n = 0
for i in range(10000):
    datum = int(np.random.normal(500, 25))
    #datum, bin_number, bin_content = bin_data(bin_edges, datum)
    
    bin_number = int(np.histogram(datum, bins=bin_edges)[0].argmax())
    #print(bin_number)
    old_bin_content = h_ds.data[f'binned_ch{ch:02d}'][bin_number]
    patches = {
        f'binned_ch{ch:02d}': [ (bin_number, old_bin_content+1) ]
    }
    
    h_ds.patch(patches)
    
    n+=1

    
    
    
    
    
    #deltaT = time.time() - tlast
    tnow = time.time()
    deltaT = tnow - tlast
    if deltaT > 0.1:
        #print(deltaT)
        tlast = tnow
        rate_ds.stream({'t':[tnow - t0], 'rate':[n / deltaT]}, rollover=1000)
        n = 0
        push_notebook(h)  

In [ ]:
for i in range(100000):
    event_counter += 1
    
    data = 250 + np.random.normal(scale=5, size=n_ch)
    steps.data_source.data['pedestals'] = data
    datastream.stream({'time': [event_counter], 
                   'adc0':[data[0]], 
                   'adc1':[data[1]]}, 
                   rollover=100)    
    
    #data_dict['pedestals'] = 250 + np.random.normal(scale=5, size=n_ch)
    #data.stream(data_dict)
    push_notebook(t)
    
    time.sleep(np.abs(np.random.normal(0.05, scale=0.01)))

In [ ]:
bin_numbers = np.array(range(len(bin_centers)), dtype=int)

type(bin_numbers)
type(bin_numbers[0])

In [ ]:
#%%timeit 

bin_content = np.histogram(np.random.normal(5, 0.1, 1000), bins=bin_edges)[0]
bin_cut = bin_content != 0

sl_cont = bin_content[bin_cut]
sl_bins = bin_numbers[bin_cut]
print(bin_content, bin_cut, bin_numbers)
print(sl_bins, sl_cont)

In [ ]:
ll = []
for i,j in zip(sl_bins, sl_cont):
    ll += [(i,j)]


#sl_bins = [2]
#sl_cont = [5]
patches = {
    #f'binned_ch{ch:02d}': list(zip(sl_bins, sl_cont))
    f'binned_ch{ch:02d}': ll
}
    
h_ds.patch(patches)

#push_notebook(h)

In [ ]:
ll = []

for i,j in zip(sl_bins, sl_cont):
    ll += [(i,j)]

In [ ]:
a = [(4, 517), (5, 483)]


In [ ]:
type(sl_bins[0])

In [ ]:
help(silayer.client)